In [19]:
!pip install --upgrade pip
!pip install sentence-transformers langchain chromadb pypdf faiss-cpu  langchain_community scikit-learn matplotlib seaborn numpy mistralai langchain-mistralai langchain_classic

In [20]:
from google.colab import files
uploaded = files.upload()


Saving Day-10.pdf to Day-10 (3).pdf


In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS,Chroma
from langchain_classic.schema import Document
import os

loader = PyPDFLoader("/content/Day-10.pdf")
docs = loader.load()


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(f"Loaded {len(docs)} docs, split into {len(chunks)} chunks")
# inspect one
print(chunks[0].page_content[:400])

Loaded 11 docs, split into 22 chunks
preencoded.png
Day 10
Vector Databases


In [22]:
print("Creating embeddings... this may take a moment.")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

Creating embeddings... this may take a moment.


In [23]:
import os
from getpass import getpass

if not os.getenv("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = getpass("Enter your Mistral API Key: ")

In [24]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-tiny")

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
""")
document_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, document_chain)

In [26]:
user_query = "what is vector database?"
response = rag_chain.invoke({"input": user_query})

print("\n--- RESPONSE ---")
print(response["answer"])


--- RESPONSE ---
A vector database is a technology optimized to store numerical representations of text, called embeddings, which are high-dimensional vectors. These databases are essential for enabling AI systems to understand and retrieve information based on meaning, rather than just keywords, and are fundamental to the next generation of intelligent applications.
